The original implementation: https://github.com/MHersche/eegnet-based-embedded-bci

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import mne
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sys
from pathlib import Path

import torch
from torch import nn
from torch.utils.data import DataLoader

current_directory = Path().cwd()
parent_directory = current_directory.parent
sys.path.insert(0, str(parent_directory))

from modules.dataloader import CreateIDs, DataSet
from modules.architecture import EEGNet_lowpower
from training_loop import TrainingLoop

sns.set_style('whitegrid')

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
DOWNSAMPLING_FACTOR = 2
NUMBER_OF_CHANNELS = 64
N_SAMPLES_PER_RECORDING = 5
TASK_MODE = 'all_tasks'

# Model

* Ns - number of input samples in time domain
* Nch - number of EEG channels
* Ncl - number of classes
* Nf - filter size of first temporal filter
* Np - pooling length
</br></br>
* n - number of filters
* p - padding strategy

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print('Using {} device'.format(device))

Using cpu device


In [6]:
EEGNet = EEGNet_lowpower(task_mode=TASK_MODE)
if torch.cuda.device_count() > 1:
    print('Using {} GPUs'.format(torch.cuda.device_count()))
    EEGNet = nn.DataParallel(EEGNet)
else:
    print(f'Loading the model to {device}')
print(EEGNet)

Loading the model to cpu
EEGNet_lowpower(
  (temp_conv): TemporalConvolution(
    (conv1): Conv2d(1, 8, kernel_size=(1, 64), stride=(1, 1), padding=same)
    (conv2): Conv2d(8, 8, kernel_size=(64, 1), stride=(1, 1), padding=same)
  )
  (batch_norm_1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (depthwise_conv): DepthWiseConvolution(
    (conv1): Conv2d(8, 8, kernel_size=(64, 1), stride=(1, 1), padding=valid, groups=8)
    (conv2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), padding=valid)
  )
  (batch_norm_2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu_1): ELU(alpha=1.0)
  (avg_pool_1): AvgPool2d(kernel_size=(1, 8), stride=8, padding=0)
  (separable_conv): Conv2d(16, 16, kernel_size=(16, 1), stride=(1, 1), padding=same)
  (batch_norm_3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (elu_2): ELU(alpha=1.0)
  (avg_pool_2): AvgPool2d(kernel_size=(1, 8), stride=(1, 8)

In [7]:
X = torch.rand(1,1,64,480, device=device)
output = EEGNet(X)
output

tensor([[0.1414, 0.1920, 0.1864, 0.3120, 0.1682]], grad_fn=<SoftmaxBackward0>)

## Data loader

In [8]:
dataset_path = ''

train_ids, val_ids, test_ids = CreateIDs(dataset_path=dataset_path,
n_samples_per_recording=N_SAMPLES_PER_RECORDING, task_mode=TASK_MODE, split_ratio=(0.7,0.2,0.1)).create()

Found 109 patient's folders
Data will be split with ratio 70.0% train, 20.0% val, 10.0% test
Splitting data into train, val and test set according to recordings, val and test will use ceil int
Checking train IDs


  0%|          | 0/4895 [00:00<?, ?it/s]

Checking validation IDs


  0%|          | 0/1635 [00:00<?, ?it/s]

Checking test IDs


  0%|          | 0/1090 [00:00<?, ?it/s]

Created 4245 train, 1350 validation, and 945 test IDs


Dataset modes:
1. rest_unrest - predicts T0 vs rest (2 classes), uses all runs
2. left_right - predicts T0-T1-T2 (3 classes), uses runs 3,4,7,8,11,12, predicts left (T1) first or right first(T2)
3. upper_lower - predicts T0-T1-T2 (3 classes), uses runs 5,6,9,10,13,14
4. all tasks - predicts 5 classes (rest, left, right, both, feet)

In [9]:
training_dataset = DataSet(list_IDs=train_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)
validation_dataset = DataSet(list_IDs=val_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)
test_dataset = DataSet(list_IDs=test_ids, dataset_path=dataset_path, sample_length=480, task_mode=TASK_MODE)

training_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)
validation_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)
test_data_loader = DataLoader(training_dataset, batch_size=10,shuffle=True)

for x_train,y in training_data_loader:
    break

for x_test,y in test_data_loader:
    break
y

tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])

# Training

6 minutes per epoch

In [10]:
save_path = ''

torch.cuda.empty_cache()
EEGNet = EEGNet_lowpower(task_mode=TASK_MODE).to(device)

#criterion = nn.BCELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(EEGNet.parameters(), lr=0.001)

training_loop = TrainingLoop(
    model=EEGNet,
    device=device,
    criterion=criterion,
    optimizer=optimizer,
    training_DataLoader=training_data_loader,
    epochs=2,
    save_path=save_path
)

training_losses, validation_losses, lr_rates = training_loop.run_training()

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
out = EEGNet(x_train)
print('Checking training data:\n')
print(out.detach().cpu().numpy())
print(y)

print('\n\n')
print('Test data:\n')
out = EEGNet(x_test)
print(out.detach().cpu().numpy())
print(y)

Checking training data:

[[9.99433100e-01 1.80345756e-04 1.78987699e-04 1.00801830e-04
  1.06763015e-04]
 [9.96871293e-01 7.78718037e-04 6.93718670e-04 9.11938667e-04
  7.44326739e-04]
 [9.98851299e-01 3.54945456e-04 2.94102152e-04 2.17172113e-04
  2.82415102e-04]
 [9.98517573e-01 4.17233416e-04 4.20516648e-04 2.30175050e-04
  4.14459530e-04]
 [9.94980395e-01 1.41474023e-03 1.21363089e-03 1.11888011e-03
  1.27233227e-03]
 [9.97337639e-01 8.59745138e-04 6.36652694e-04 5.84352703e-04
  5.81616419e-04]
 [9.96548831e-01 1.00441801e-03 5.84950321e-04 7.08344625e-04
  1.15337432e-03]
 [9.97514248e-01 7.70908839e-04 6.12541335e-04 3.54911404e-04
  7.47310580e-04]
 [9.98891532e-01 3.60714621e-04 2.20241083e-04 2.50124111e-04
  2.77285988e-04]
 [9.98627186e-01 4.28594387e-04 3.28698690e-04 2.74863385e-04
  3.40755156e-04]]
tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0.,

In [13]:
for x_test,y_test in test_data_loader:
    break

In [15]:
y_test

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.]])

In [16]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# For test data
test_predictions = EEGNet(x_test).detach().cpu().numpy()
test_predictions = to_class_indices(test_predictions)  # Convert predictions to class indices
test_labels = to_class_indices(y_test.numpy())  # Convert labels to class indices

# Calculate metrics for test data
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions, average='macro')  # Use 'macro' or 'weighted' for multi-class
test_recall = recall_score(test_labels, test_predictions, average='macro')  # Use 'macro' or 'weighted' for multi-class
test_f1 = f1_score(test_labels, test_predictions, average='macro')  # Use 'macro' or 'weighted' for multi-class

print('Test Data Metrics:')
print(f'Accuracy: {test_accuracy}')
print(f'Precision: {test_precision}')
print(f'Recall: {test_recall}')
print(f'F1 Score: {test_f1}')


NameError: name 'to_class_indices' is not defined

In [12]:
out = EEGNet(x_test)
print(out)
print(y)

tensor([[9.9816e-01, 5.3798e-04, 3.4520e-04, 3.5137e-04, 6.0680e-04],
        [9.9845e-01, 5.1368e-04, 3.0429e-04, 2.4003e-04, 4.9191e-04],
        [9.9677e-01, 9.3339e-04, 7.1715e-04, 7.0674e-04, 8.7046e-04],
        [9.9761e-01, 6.8993e-04, 5.2543e-04, 4.4516e-04, 7.2956e-04],
        [9.9843e-01, 4.8130e-04, 3.0209e-04, 3.6396e-04, 4.2766e-04],
        [9.9811e-01, 6.5541e-04, 4.9784e-04, 3.4570e-04, 3.9233e-04],
        [9.9969e-01, 1.0241e-04, 7.1644e-05, 6.0103e-05, 7.7587e-05],
        [9.9883e-01, 4.2424e-04, 2.3859e-04, 2.0950e-04, 2.9622e-04],
        [9.9703e-01, 1.0667e-03, 7.9589e-04, 4.8310e-04, 6.2438e-04],
        [9.9726e-01, 7.8234e-04, 9.5603e-04, 4.6460e-04, 5.3862e-04]],
       grad_fn=<SoftmaxBackward0>)
tensor([[1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1.,